In [6]:
import telebot
from bs4 import BeautifulSoup
import requests
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import datetime
import time
import random

#Getting access to Google sheets and Google drive APIs
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('newsscraper-420116-8754ebe01d08.json', scope) # You have to register you project and get the json key
client = gspread.authorize(creds)
table_name = 'Первый тестовый документ' # Name of your google sheet table
sheet = client.open(table_name).sheet1

# Setting your Telegram bot
API_TOKEN = '6920880710:AAH1qx0NbfN88XbK-E_DEsqFCv6AmukTvGg' # You have to register your bot through Botfather and get the API
bot = telebot.TeleBot(API_TOKEN)

# Getting the news headlines from the news website
def get_news_headlines(url, object_to_find):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    headlines = [headline.text for headline in soup.find_all(object_to_find[0], class_ = object_to_find[1])] # We are looking for a specific element on website with the specific name of class
    return headlines

# To have all the commands displayed in pop-window write them all to Botfather

# Handler of '/start' command
@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.reply_to(message, "Привет! Я бот, который собирает заголовки новостей.\nВыберите комманду из списка команд")

user_in_process = {} #Users, that are in process of getting the results /getnews command

# Handler of '/getnews' command
@bot.message_handler(commands=['getnews'])
def get_news(message):
    user_id = message.from_user.id
    current_time = time.time()
    #Checking if the user has already clicked the command that hasn't been executed yet
    if user_in_process.get(user_id, False):
        bot.reply_to(message, "Ваш предыдущий запрос все еще обрабатывается. Пожалуйста, подождите.")
        return
    user_in_process[user_id] = True #Flagging the user that he has active command that is still executing

    delay = 0.1 #Delay to prevent Google APIs overload with requests
    while True:
        try:
            source_website = 'https://meduza.io/' # Our news website
            objects = ['span', 'BlockTitle-module-first'] #What we are looking for [type of element, its class name]
            headlines = get_news_headlines(source_website, objects)
            delta = datetime.timedelta(hours=3) #Setting our TimeZone
            recording_time = (datetime.datetime.now() + delta).strftime('%d-%m-%Y %H:%M:%S') #Adjusting datetime format
            for headline in headlines:
                sheet.append_row([recording_time, headline])
            bot.reply_to(message, "Заголовки новостей добавлены в таблицу")
            del user_in_process[user_id]
            break
        except Exception as e:
            if e.response.status_code == 429: #We might get the error of exceeded limit of requests to Google API
                #Waiting for the delay
                time.sleep(delay)
                #Increasing the delay time for next attempt
                delay *= 2
                #Adding randomness to avoid accidental synchronization among multiple users
                delay += random.uniform(0, 1)
            else:
                bot.reply_to(message, f"Произошла ошибка: {e}")
                del user_in_process[user_id]
                break

#Starting our bot
bot.polling()
